<a href="https://colab.research.google.com/github/Abi20601/Image-Video-Processing/blob/main/IVA_R1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Canny edge detection and Masking**

In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [ ]:
from google.colab import drive

In [ ]:
paths = glob.glob('/content/drive/MyDrive/Fish_dataset/*.jpg',recursive=True)

In [ ]:
orig = np.array([np.asarray(Image.open(img)) for img in paths])
plt.figure(figsize=(25,25))

for i, img in enumerate(orig[0:5]):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(img)

plt.show()

In [ ]:
gray = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in orig])
plt.figure(figsize=(25,25))

for i, img in enumerate(gray[0:5]):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_GRAY2RGB))

plt.show()

**Thresholding**

The thresholding capacity included here is mean of all the pixels present in the images.

In [ ]:
thresh = [cv2.threshold(img, np.mean(img), 255, cv2.THRESH_BINARY_INV)[1] for img in gray]
thresh[0].shape

In [ ]:
plt.figure(figsize=(25,25))

for i, threshimg in enumerate(thresh[0:5]):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(threshimg,cmap='gray')

plt.show()

**Edge Detection**

Here edgedetction is done using canny edge detection. And Dilation is used for removing noises from the image.

In [ ]:
edges = [cv2.dilate(cv2.Canny(img, 0, 255), None) for img in thresh]

In [ ]:
plt.figure(figsize=(25,25))
# plt.suptitle("Edges", fontsize=150)

for i, edge in enumerate(edges[0:5]):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  # plt.grid(False)
  plt.imshow(cv2.cvtColor(edge, cv2.COLOR_GRAY2RGB))

plt.show()

**Masking**

Masking is done for creating a mask by finding the contour regions of the binary image.After finding the contours in the input image, we will find the largest contour which will be the object of interest.

In [ ]:
masked = []
segmented = []
for i, img in enumerate(edges):
  cnt = sorted(cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2], key=cv2.contourArea)[-1]
  mask = np.zeros((256,256), np.uint8)
  masked.append(cv2.drawContours(mask, [cnt],-1, 255, -1))

In [ ]:
plt.figure(figsize=(25,25))

for i, maskimg in enumerate(masked[0:5]):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(maskimg, cmap='gray')

plt.show()

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
from skimage.feature import canny
from scipy import ndimage as ndi

plt.figure(figsize=(25,5))
path = "/content/drive/MyDrive/Fish_dataset/"

for file_name in os.listdir(path):
    if file_name.endswith(".jpg"):
        image = cv2.imread(os.path.join(path, file_name))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply Canny edge detection algorithm
        edges = canny(gray / 255.)

        # Plot the Canny edge detection output
        fig, ax = plt.subplots(figsize=(4, 3))
        ax.imshow(edges, cmap=plt.cm.gray, interpolation='nearest')
        ax.axis('off')
        ax.set_title('Canny detector')

        # Fill the holes in the Canny output
        fill_coins = ndi.binary_fill_holes(edges)

        fig, ax = plt.subplots(figsize=(4, 3))
        ax.imshow(fill_coins, cmap=plt.cm.gray, interpolation='nearest')
        ax.axis('off')
        ax.set_title('Filling the holes')

Segmentation

It is done by taking a bitwise_and of both masked image and original image.The the final segmented image is converted back to colour image.

In [ ]:
for i,mas in enumerate(edges(0:5)):
  dst = cv2.bitwise_and(orig[i], orig[i], mask=mas)
  segmented.append(cv2.cvtColor(dst, cv2.COLOR_BGR2RGB))

In [ ]:
plt.figure(figsize=(25,25))

for i, segimg in enumerate(segmented[0:5]):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(cv2.cvtColor(segimg, cv2.COLOR_BGR2RGB))

plt.show()

## **Otsu's Method**

In [ ]:
plt.figure(figsize=(25,5))
for i in range(1, 6):
    # Read image and convert to RGB format
    img = cv2.imread(f"/content/drive/MyDrive/Fish_dataset/0{i}.jpg")
    b, g, r = cv2.split(img)
    rgb_img = cv2.merge([r, g, b])

    # Convert image to grayscale and apply Otsu's thresholding
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # Apply morphological closing to remove noise and fill gaps
    kernel = np.ones((2, 2), np.uint8)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Apply distance transform to find sure foreground and unknown regions
    sure_bg = cv2.dilate(closing, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(sure_bg, cv2.DIST_L2, 3)
    ret, sure_fg = cv2.threshold(dist_transform, 0.1*dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Apply watershed segmentation
    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0
    markers = cv2.watershed(img, markers)
    img[markers == -1] = [255, 0, 0]


    # Display input image and thresholded image
    plt.subplot(2, 5, i), plt.imshow(rgb_img)
    plt.title(f"Image{i}"), plt.xticks([]), plt.yticks([])
    plt.subplot(2, 5, i+5), plt.imshow(thresh, 'gray')
    plt.title(f"Otsu's{i}"), plt.xticks([]), plt.yticks([])

**CHROMATICITY SEGMENTATION**

In [ ]:
# Define the Gaussian Distribution
def gaussian(p,mean,std):
    return np.exp(-(p-mean)**2/(2*std**2))*(1/(std*((2*np.pi)**0.5)))

In [ ]:
from google.colab.patches import cv2_imshow
from skimage.filters import threshold_multiotsu
for i in range(5):
  patch = orig[i][250:300,300:350,:]

  patch_R = patch[:,:,0]*1.0/patch.sum(axis=2)
  patch_G = patch[:,:,1]*1.0/patch.sum(axis=2)
  # For the R axis
  std_patch_R = np.std(patch_R.flatten())
  mean_patch_R = np.mean(patch_R.flatten())
  # For the G axis
  std_patch_G = np.std(patch_G.flatten())
  mean_patch_G = np.mean(patch_G.flatten())

  prob_R = gaussian(patch_R,mean_patch_R,std_patch_R)
  prob_G = gaussian(patch_G,mean_patch_G,std_patch_G)
  prob=prob_R * prob_G

  plt.figure(figsize=(5,5))
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(prob)

plt.show()

# Shape Analysis

**Fourier Transformation and Reconstruction**

In [ ]:
import numpy as np
from scipy import fftpack
import matplotlib.pyplot as plt
from PIL import Image
import PIL.Image as Image

# Load the shape as a binary image
shape = np.array(Image.open('/content/drive/MyDrive/Fish_dataset/03.jpg').convert('L')) > 128


image = Image.open('/content/drive/MyDrive/Fish_dataset/03.jpg')
print(f'Bit Depth: {image.mode}')
print(f'Color Mode: {image.info.get("color_mode")}')

# Compute the contour of the shape
contour = np.zeros_like(shape, dtype=np.uint8)
contours, _ = cv2.findContours(shape.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cv2.drawContours(contour, contours, 0, 255, thickness=1)

# # Compute the Fourier descriptors of the shape
fourier_coeffs = fftpack.fft(contour.flatten())
descriptor = np.abs(fourier_coeffs[:len(fourier_coeffs)//2])

# # Reconstruct the contour from the Fourier descriptors
reconstructed_coeffs = np.zeros_like(fourier_coeffs)
reconstructed_coeffs[:len(descriptor)] = descriptor
reconstructed_coeffs[-len(descriptor):] = descriptor[::-1]
reconstructed_contour = fftpack.ifft(reconstructed_coeffs).real.reshape(contour.shape)


# Normalize the reconstructed contour to have the same range of pixel values as the original shape
reconstructed_contour = (reconstructed_contour - reconstructed_contour.min()) / (reconstructed_contour.max() - reconstructed_contour.min()) * 255

# Plot the original shape and the reconstructed shape
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(shape, cmap='gray')
ax[0].set_title('Original Shape')
ax[1].imshow(reconstructed_contour, cmap='gray')
ax[1].set_title('Reconstructed Shape')
plt.show()

In [ ]:
import cv2
img = plt.imread("/content/drive/MyDrive/Fish_dataset/04.jpg")
img = cv2.bitwise_not(img)
grayImage = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
ret, img_bin = cv2.threshold(grayImage, 127, 255, cv2.THRESH_BINARY)
plt.xticks([])
plt.yticks([])
plt.imshow(img_bin,cmap="gray")

for i, row in enumerate(img_bin):
    for j, value in enumerate(row):
        if value == 255:
            start_point = [i, j]
            print(start_point, value)
            break
    else:
        continue
    break

b = np.array([start_point],dtype=int)
c = np.array([[start_point[0],start_point[1]-1]],dtype=int)

#Different Sequence for different position of c
seq = [[0,7,6,5,4,3,2,1],
        [1,0,7,6,5,4,3,2],
        [2,1,0,7,6,5,4,3],
        [3,2,1,0,7,6,5,4],
        [4,3,2,1,0,7,6,5],
        [5,4,3,2,1,0,7,6],
        [6,5,4,3,2,1,0,7],
        [7,6,5,4,3,2,1,0]]

while True:
    latest_b = b[len(b)-1]
    latest_c = c[len(c)-1]
    diff = b[len(b)-1] - c[len(c)-1]
    if (diff == [0,-1]).all():
        seq_no = 0
    if (diff == [1,-1]).all():
        seq_no = 1
    if (diff == [1,0]).all():
        seq_no = 2
    if (diff == [1,1]).all():
        seq_no = 3
    if (diff == [0,1]).all():
        seq_no = 4
    if (diff == [-1,1]).all():
        seq_no = 5
    if (diff == [-1,0]).all():
        seq_no = 6
    if (diff == [-1,-1]).all():
        seq_no = 7

    for i in seq[seq_no]:
        if i==0 and (latest_b[1]+1<img_bin.shape[1]) and (img_bin[latest_b[0]][latest_b[1]+1] == 255):
            new_b = [latest_b[0], latest_b[1]+1]
            new_c = [latest_b[0]-1,latest_b[1]+1]
            break

        if i==1 and (latest_b[0]-1>=0)and(latest_b[1]+1<img_bin.shape[1]) and (img_bin[latest_b[0]-1][latest_b[1]+1] == 255):
            new_b = [latest_b[0]-1, latest_b[1]+1]
            new_c = [latest_b[0]-1,latest_b[1]]
            break

        if i==2 and (latest_b[0]-1>=0) and (img_bin[latest_b[0]-1][latest_b[1]] == 255):
            new_b = [latest_b[0]-1, latest_b[1]]
            new_c = [latest_b[0]-1, latest_b[1]-1]
            break

        if i==3 and (latest_b[0]-1>=0)and(latest_b[1]-1>=0) and (img_bin[latest_b[0]-1][latest_b[1]-1] == 255):
            new_b = [latest_b[0]-1, latest_b[1]-1]
            new_c = [latest_b[0], latest_b[1]-1]
            break


        if i==4 and (latest_b[1]-1>=0) and (img_bin[latest_b[0]][latest_b[1]-1] == 255):
            new_b = [latest_b[0], latest_b[1]-1]
            new_c = [latest_b[0]+1, latest_b[1]-1]
            break

        if i==5 and (latest_b[0]+1<img_bin.shape[0])and(latest_b[1]-1>=0) and (img_bin[latest_b[0]+1][latest_b[1]-1] == 255):
            new_b = [latest_b[0]+1, latest_b[1]-1]
            new_c = [latest_b[0]+1, latest_b[1]]
            break

        if i==6 and (latest_b[0]+1<img_bin.shape[0]) and (img_bin[latest_b[0]+1][latest_b[1]] == 255):
            new_b = [latest_b[0]+1, latest_b[1]]
            new_c = [latest_b[0]+1, latest_b[1]+1]
            break

        if i==7 and (latest_b[0]+1<img_bin.shape[0])and(latest_b[1]+1<img_bin.shape[1]) and (img_bin[latest_b[0]+1][latest_b[1]+1] == 255):
            new_b = [latest_b[0]+1, latest_b[1]+1]
            new_c = [latest_b[0], latest_b[1]+1]
            break


    b = np.append(b,[new_b],axis=0)
    c = np.append(c,[new_c],axis=0)
    if (b[0] == b[len(b)-1]).all():
        break

In [ ]:
boundary = np.zeros((img_bin.shape[0],img_bin.shape[1]),dtype=int)
for i in range(0, b.shape[0]):
    boundary[b[i][0], b[i][1]] = 255

plt.xticks([])
plt.yticks([])
plt.imshow(boundary,cmap="gray")